In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My Drive/PROJECT - NLP/semantic-segmentation/data/text/files

/content/drive/My Drive/PROJECT - NLP/semantic-segmentation/data/text/files


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('file2.csv')

In [0]:
print(len(df[df['isLong'] == True]))
df = df[df['isLong'] == False]

39


In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 47.1MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 3.7MB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b8847938fcd860165e3d4b32cf8e44d845f66ee2760ee55ad09d5624c4655f22
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [0]:
from sklearn.model_selection import train_test_split
sentences,sentences1,labels,labels1 = train_test_split(df.Sentence.values,df.Label.values,test_size=0.1)

In [0]:
max_len = 0
for sent in sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len,len(input_ids))
print('Max sentence length: ', max_len)

Max sentence length:  220


In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5,random_state=42,shuffle=True)
kf.get_n_splits(sentences)
train_d = []
test_d = []
for train_index,test_index in kf.split(sentences,labels):
  train_d.append(train_index)
  test_d.append(test_index)

In [0]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [0]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 220,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  66,Sub-s.(3) of Section 69,Section
Token IDs: tensor([ 101, 5764, 1010, 4942, 1011, 1055, 1012, 1006, 1017, 1007, 1997, 2930,
        6353, 1010, 2930,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  

In [0]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 7, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [0]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [0]:
CUDA_LAUNCH_BLOCKING=1

In [0]:
import random
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
batch_size = 32
epochs = 10
training_stats = []

for i in range(kf.get_n_splits()):
  train_dataset = TensorDataset(input_ids[train_d[i]], attention_masks[train_d[i]], labels[train_d[i]])
  test_dataset = TensorDataset(input_ids[test_d[i]], attention_masks[test_d[i]], labels[test_d[i]])
 
  train_dataloader = DataLoader(
              train_dataset,  # The training samples.
              sampler = RandomSampler(train_dataset), # Select batches randomly
              batch_size = batch_size # Trains with this batch size.
          )
  # For validation the order doesn't matter, so we'll just read them sequentially.
  test_dataloader = DataLoader(
              test_dataset, # The validation samples.
              sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )

  total_steps = len(train_dataloader) * epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

  model.load_state_dict(torch.load('/content/drive/My Drive/PROJECT - NLP/semantic-segmentation/data/pretrained-BERT-param'))
  total_t0 = time.time()

  for epoch_i in range(0, epochs):
    
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        
          loss, logits = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
          
          total_train_loss += loss.item()
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          optimizer.step()
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in test_dataloader:
          
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          
          with torch.no_grad():        

              (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
              
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      avg_val_loss = total_eval_loss / len(test_dataloader)
      
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'KFold' : i,
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  torch.save(model.state_dict(), '/content/drive/My Drive/PROJECT - NLP/semantic-segmentation/model_Kfold/model'+str(i))

======== Epoch 1 / 10 ========
Training...


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


  Batch    40  of    211.    Elapsed: 0:00:26.
  Batch    80  of    211.    Elapsed: 0:00:51.
  Batch   120  of    211.    Elapsed: 0:01:16.
  Batch   160  of    211.    Elapsed: 0:01:41.
  Batch   200  of    211.    Elapsed: 0:02:06.

  Average training loss: 1.28
  Training epcoh took: 0:02:13

Running Validation...
  Accuracy: 0.59
  Validation Loss: 1.16
  Validation took: 0:00:10
======== Epoch 2 / 10 ========
Training...
  Batch    40  of    211.    Elapsed: 0:00:25.
  Batch    80  of    211.    Elapsed: 0:00:50.
  Batch   120  of    211.    Elapsed: 0:01:16.
  Batch   160  of    211.    Elapsed: 0:01:41.
  Batch   200  of    211.    Elapsed: 0:02:06.

  Average training loss: 0.95
  Training epcoh took: 0:02:12

Running Validation...
  Accuracy: 0.61
  Validation Loss: 1.12
  Validation took: 0:00:10
======== Epoch 3 / 10 ========
Training...
  Batch    40  of    211.    Elapsed: 0:00:25.
  Batch    80  of    211.    Elapsed: 0:00:50.
  Batch   120  of    211.    Elapsed: 0:01:1

In [0]:
df_stats = pd.DataFrame(data = training_stats)

In [0]:
df_stats[df_stats['epoch'] == 10]

,KFold,epoch,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
9,0,10,0.028015,2.352577,0.614872,0:02:12,0:00:10
19,1,10,0.020314,2.285149,0.618410,0:02:11,0:00:10
29,2,10,0.023844,2.131364,0.640816,0:02:11,0:00:10
39,3,10,0.026888,2.261385,0.630862,0:02:11,0:00:10
49,4,10,0.024394,2.197891,0.634122,0:02:11,0:00:10


In [0]:
import os

In [0]:
%cd /content/drive/My Drive/PROJECT - NLP/semantic-segmentation/model_Kfold/

/content/drive/My Drive/PROJECT - NLP/semantic-segmentation/model_Kfold


In [0]:
input_ids = []
attention_masks = []

for sent in sentences1:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 220,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels1)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  66,Sub-s.(3) of Section 69,Section
Token IDs: tensor([  101,  1996,  4428,  2486,  2029,  7796,  2015,  1996, 11372,  1997,
         1037,  3327,  2095,  2011, 20295,  2007,  1996, 12433,  2003,  5664,
         2013,  1996,  2028,  2029,  5201,  2000,  1996, 11372,  1997,  1996,
         3025,  2086,  1998,  2045,  2003,  2053, 13717,  2090,  1996,  4428,
         2749,  2029,  2024,  4846,  1999,  1996,  3919,  5142,  2076,  1996,
         2195,  2086,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
       

In [0]:
from sklearn.metrics import classification_report
Report = {}
l = len(input_ids)

for fil in os.listdir():
  model.load_state_dict(torch.load(fil))
  preds_labels = []
  for i in range(l):
    t = model(input_ids[i].unsqueeze(0).cuda(), attention_mask=attention_masks[i].unsqueeze(0).cuda(),labels=labels[i].unsqueeze(0).cuda())[1]
    preds_labels.append(np.argmax(t.detach().cpu().numpy()))
  dic = classification_report(labels,preds_labels, labels=[0,1,2,3,4,5,6], target_names=['Argument', 'Facts', 'Precedent', 'Ratio of the decision','Ruling by Lower Court', 'Ruling by Present Court', 'Statute'], digits=2, output_dict=True, zero_division='warn')
  Report[fil] = dic

In [0]:
df_Report_model0 = pd.DataFrame(data = Report['model0'])
df_Report_model1 = pd.DataFrame(data = Report['model1'])
df_Report_model2 = pd.DataFrame(data = Report['model2'])
df_Report_model3 = pd.DataFrame(data = Report['model3'])
df_Report_model4 = pd.DataFrame(data = Report['model4'])

In [0]:
df_Report_model0

,Argument,Facts,Precedent,Ratio of the decision,Ruling by Lower Court,Ruling by Present Court,Statute,accuracy,macro avg,weighted avg
precision,0.594937,0.692308,0.669065,0.657635,0.363636,0.500000,0.620690,0.648822,0.585467,0.645824
recall,0.505376,0.682464,0.560241,0.743733,0.242424,0.647059,0.654545,0.648822,0.576549,0.648822
f1-score,0.546512,0.687351,0.609836,0.698039,0.290909,0.564103,0.637168,0.648822,0.576274,0.644453
support,93.000000,211.000000,166.000000,359.000000,33.000000,17.000000,55.000000,0.648822,934.000000,934.000000


In [0]:
df_Report_model1

,Argument,Facts,Precedent,Ratio of the decision,Ruling by Lower Court,Ruling by Present Court,Statute,accuracy,macro avg,weighted avg
precision,0.578313,0.669903,0.650000,0.639024,0.312500,0.521739,0.625000,0.632762,0.570926,0.627408
recall,0.516129,0.654028,0.548193,0.729805,0.151515,0.705882,0.636364,0.632762,0.563131,0.632762
f1-score,0.545455,0.661871,0.594771,0.681404,0.204082,0.600000,0.630631,0.632762,0.559745,0.626721
support,93.000000,211.000000,166.000000,359.000000,33.000000,17.000000,55.000000,0.632762,934.000000,934.000000


In [0]:
df_Report_model2

,Argument,Facts,Precedent,Ratio of the decision,Ruling by Lower Court,Ruling by Present Court,Statute,accuracy,macro avg,weighted avg
precision,0.581081,0.688679,0.682171,0.646081,0.347826,0.600000,0.672727,0.648822,0.602652,0.645839
recall,0.462366,0.691943,0.530120,0.757660,0.242424,0.705882,0.672727,0.648822,0.580446,0.648822
f1-score,0.514970,0.690307,0.596610,0.697436,0.285714,0.648649,0.672727,0.648822,0.586631,0.642847
support,93.000000,211.000000,166.000000,359.000000,33.000000,17.000000,55.000000,0.648822,934.000000,934.000000


In [0]:
df_Report_model3

,Argument,Facts,Precedent,Ratio of the decision,Ruling by Lower Court,Ruling by Present Court,Statute,accuracy,macro avg,weighted avg
precision,0.585366,0.704433,0.717391,0.638695,0.230769,0.750000,0.679245,0.658458,0.615129,0.652223
recall,0.516129,0.677725,0.596386,0.763231,0.090909,0.705882,0.654545,0.658458,0.572115,0.658458
f1-score,0.548571,0.690821,0.651316,0.695431,0.130435,0.727273,0.666667,0.658458,0.587216,0.650849
support,93.000000,211.000000,166.000000,359.000000,33.000000,17.000000,55.000000,0.658458,934.000000,934.000000


In [0]:
df_Report_model4

,Argument,Facts,Precedent,Ratio of the decision,Ruling by Lower Court,Ruling by Present Court,Statute,accuracy,macro avg,weighted avg
precision,0.518987,0.678571,0.645833,0.630332,0.444444,0.571429,0.722222,0.633833,0.601688,0.630669
recall,0.440860,0.630332,0.560241,0.740947,0.242424,0.705882,0.709091,0.633833,0.575683,0.633833
f1-score,0.476744,0.653563,0.600000,0.681178,0.313725,0.631579,0.715596,0.633833,0.581769,0.628297
support,93.000000,211.000000,166.000000,359.000000,33.000000,17.000000,55.000000,0.633833,934.000000,934.000000
